# 📐 Vectors

## Overview

### System.Numerics : SIMD

https://devblogs.microsoft.com/dotnet/dotnet-8-hardware-intrinsics

- This is a SIMD-enabled (Single Instruction Multiple Data) structure in System.Numerics
- It represents a fixed-size vector of primitive numeric types (like int, float, double)
- The size is determined by the hardware's SIMD capabilities
- It's primarily designed for high-performance numeric computations
- Software Fallback

In [ ]:
using System.Numerics;

Console.WriteLine($"{Vector<int>.Count}");
Console.WriteLine($"{Vector<float>.Count}");

Console.WriteLine($"{Vector<double>.Count}");
Console.WriteLine($"{Vector<long>.Count}");

// Vector for decimal, bool not possible; why ?

### System.Runtime.Intrinsics

- https://devblogs.microsoft.com/dotnet/dotnet-8-hardware-intrinsics/
- [2019] .NET Core 3: https://devblogs.microsoft.com/dotnet/hardware-intrinsics-in-net-core
    - Vector128, Vector256, Sse, Sse2, Sse3, Sse41, Sse42, Avx, Avx2, Fma, Bmi1, Bmi2, Lzcnt, Popcnt, Aes, and Pclmul for x86/x64
- [2020] .NET 5: https://devblogs.microsoft.com/dotnet/announcing-net-5-0-preview-7
    - Vector64, AdvSimd, ArmBase, Dp, Rdm, Aes, Crc32, Sha1, and Sha256 for Arm/Arm64
- [2021] .NET 6: https://devblogs.microsoft.com/dotnet/performance-improvements-in-net-6
    - AvxVnni
    - System.Numerics uses System.Runtime.Intrinsics
- [2020] .NET 7: https://devblogs.microsoft.com/dotnet/performance_improvements_in_net_7
    - Vector64, Vector128, Vector256 and Vector<T> enhancements, X86Serialize
- [2021] .NET 8: https://devblogs.microsoft.com/dotnet/dotnet-8-hardware-intrinsics
    - PackedSimd, WasmBase, Vector512<T>, Avx512F, Avx512BW, Avx512CD, Avx512DQ, and Avx512Vbmi
    - Wasm and Avx512

In [ ]:
System.Runtime.Intrinsics.Vector512.IsHardwareAccelerated

In [ ]:
using System.Numerics;
using System.Runtime.Intrinsics;

Console.WriteLine($"Vector128 of int:\t{Vector128<int>.Count}");
Console.WriteLine($"Vector256 of int:\t{Vector256<int>.Count}");
Console.WriteLine($"Vector512 of int:\t{Vector512<int>.Count}");

Console.WriteLine($"Vector of int:\t\t{Vector<int>.Count}");

### Advanced Vector Extensions (AVX and AVX2)

In [ ]:
using System.Runtime.Intrinsics;
using System.Runtime.Intrinsics.X86;

var vec1 = Vector256.Create(1, 2, 3, 4, 5, 6, 7, 8);
var vec2 = Vector256.Create(8, 7, 6, 5, 4, 3, 2, 1);
Vector256<int> resultAdd = Avx2.Add(vec1, vec2);

Console.WriteLine("Addition:");
for (int i = 0; i < Vector256<int>.Count; i++)
    Console.Write(resultAdd.GetElement(i) + " ");
Console.WriteLine();

In [ ]:
using System.Runtime.Intrinsics;
using System.Runtime.Intrinsics.X86;

if (Avx2.IsSupported)

### Resources

- - https://devblogs.microsoft.com/dotnet/using-net-hardware-intrinsics-api-to-accelerate-machine-learning-scenarios

## Usage: Monte Carlo Method

- https://en.wikipedia.org/wiki/Monte_Carlo_method

### Pi

In [ ]:
using System.Numerics;

double EstimatePi()
{
    Random random = new Random();

    double totalPoints = 1_000_000_000;
    int batchSize = Vector<float>.Count;
    Vector<float> ones = new Vector<float>(1.0f);

    Span<float> xSpan = stackalloc float[batchSize];
    Span<float> ySpan = stackalloc float[batchSize];
    double insideCircle = 0;

    for (double i = 0; i < totalPoints; i += batchSize)
    {
        for (int j = 0; j < batchSize; j++)
        {
            xSpan[j] = (float)random.NextDouble();
            ySpan[j] = (float)random.NextDouble();
        }

        Vector<float> xVector = new Vector<float>(xSpan);
        Vector<float> yVector = new Vector<float>(ySpan);

        Vector<float> distanceSquared = xVector * xVector + yVector * yVector;
        Vector<int> inside = Vector.LessThanOrEqual(distanceSquared, ones);
        insideCircle += Vector.Dot(inside, Vector<int>.One);
    }

    return Math.Abs(4.0f * insideCircle / totalPoints);
}

double pi = EstimatePi();
Console.WriteLine($"Estimated value of π: {pi} and we are off by {Math.Abs(Math.PI - pi)}");

In [ ]:
using System.Numerics;
using System.Threading;
using System.Threading.Tasks;

double EstimatePiParallel()
{
    long totalPoints = 1_000_000_000;
    int batchSize = Vector<float>.Count;
    int numTasks = Environment.ProcessorCount;

    Vector<float> ones = new Vector<float>(1.0f);
    long insideCircle = 0;

    Parallel.For(0, numTasks, taskIndex =>
    {
        Random random = new Random(taskIndex);
        long taskInsideCircle = 0;
        Span<float> xSpan = stackalloc float[batchSize];
        Span<float> ySpan = stackalloc float[batchSize];

        long pointsPerTask = totalPoints / numTasks;

        for (int i = 0; i < pointsPerTask; i += batchSize)
        {
            for (int j = 0; j < batchSize; j++)
            {
                xSpan[j] = (float)random.NextDouble();
                ySpan[j] = (float)random.NextDouble();
            }

            Vector<float> xVector = new Vector<float>(xSpan);
            Vector<float> yVector = new Vector<float>(ySpan);

            Vector<float> distanceSquared = xVector * xVector + yVector * yVector;
            Vector<int> inside = Vector.LessThanOrEqual(distanceSquared, ones);
            taskInsideCircle += Vector.Dot(inside, Vector<int>.One);
        }

        Interlocked.Add(ref insideCircle, taskInsideCircle);
    });

    return Math.Abs(4.0f * insideCircle / totalPoints);
}

double pi = EstimatePiParallel();
Console.WriteLine($"Estimated value of π: {pi} and we are off by {Math.Abs(Math.PI - pi):F20}");


### Box Muller Transform

- https://en.wikipedia.org/wiki/Box%E2%80%93Muller_transform

In [ ]:
using System.Runtime.Intrinsics;
using System.Runtime.Intrinsics.X86;

static Random random = new Random();

static double GenerateGaussianNumber()  // Gaussian Random Number
{                                       // Gaussian distrubtion aka Normal distribution 
    // Box-Muller Transform
    double u1 = 1.0 - random.NextDouble();
    double u2 = 1.0 - random.NextDouble();
    return Math.Sqrt(-2.0 * Math.Log(u1)) * Math.Cos(2.0 * Math.PI * u2);
}

static Vector256<double> GenerateGaussianVectorBoxMuller()
{
    // Generate two batches of random numbers
    double[] array1 = new double[Vector256<double>.Count];
    double[] array2 = new double[Vector256<double>.Count];
    for (int i = 0; i < Vector256<double>.Count; i++)
    {
        array1[i] = 1.0 - (double)random.NextDouble();
        array2[i] = 1.0 - (double)random.NextDouble();
    }

    unsafe
    {
        fixed (double* a1Ptr = array1, a2Ptr = array2)
        {
            // Load the random numbers into vectors.
            var u1 = Avx.LoadVector256(a1Ptr);
            var u2 = Avx.LoadVector256(a2Ptr);

            // Apply the Box-Muller transform.
            var r = Avx2.Sqrt(-2.0f * Avx2.Log(u1));
            var theta = 2.0f * (float)Math.PI * u2;
            var z0 = r * Avx2.Cos(theta);
            return z0;
        }
    }
}

//MathNet.Numerics.Distributions.Normal normal
//double mean = 0,double std = .01
//normal = new MathNet.Numerics.Distributions.Normal(mean,std);
//return normal.Sample();

### Ziggurat Algorithm

__Vector Operations Required for Ziggurat__

| Operation | Purpose | Available in AVX2? | Notes |
| --- | --- | --- | --- |
| Addition/Subtraction | Basic arithmetic on vectors | ✅ Yes | Supported directly in AVX2 |
| Multiplication/Division | Scaling or normalization | ✅ Yes | Supported directly in AVX2 |
| Comparison (<, >, =) | Acceptance/rejection logic | ✅ Yes	| Available through Avx2.Compare |
| Random Number Generation | Uniform sampling in [-1,1] | ❌ No | Needs to be implemented externally |
| Absolute Value | For comparing magnitude of samples | ✅ Yes | Available via Avx.AndNot or similar |
| Logarithm (log) | For fallback sampling outside ziggurat | ❌ No | Not available natively in AVX2 |
| Exponential (exp) | For fallback in rejection sampling | ❌ No | Needs approximation or external function |

### Marsaglia Polar Method

- https://en.wikipedia.org/wiki/Marsaglia_polar_method

In [ ]:
using System.Runtime.Intrinsics;
using System.Runtime.Intrinsics.X86;

Random random = new Random();

double[] GenerateRandomArray(int size)
{
    var result = new double[size];
    for (int i = 0; i < size; i++)
        result[i] = 2.0 * random.NextDouble() - 1.0; // Generate numbers in [-1,1]
    return result;
}

IEnumerable<Vector256<double>> GenerateGaussianVectorMarsaglia(int batchSize)
{
    var zero = Vector256.Create(0.0);
    var one = Vector256.Create(1.0);
    var rsqrt2 = Vector256.Create(Math.Sqrt(2.0));
    int count = 0;

    while (count <= batchSize)
    {
        Vector256<double> x, y, s, invalidMask;

        do
        {
            // Generate random vectors
            x = Vector256.Create(GenerateRandomArray(Vector256<double>.Count));
            y = Vector256.Create(GenerateRandomArray(Vector256<double>.Count));

            // Calculate x² + y²
            s = Avx.Add(Avx.Multiply(x, x), Avx.Multiply(y, y));

            // Check if all pairs are within the unit circle
            // We'll need to check each element since we can't do a vector comparison directly
            // s.GetElement(0) >= 1.0 || s.GetElement(1) >= 1.0 || 
            // s.GetElement(2) >= 1.0 || s.GetElement(3) >= 1.0 || 
            // s.GetElement(0) == 0.0 || s.GetElement(1) == 0.0 || 
            // s.GetElement(2) == 0.0 || s.GetElement(3) == 0.0

            // Determine invalid mask
            var greaterOrEqualOne = Avx.CompareGreaterThanOrEqual(s, one);
            var equalZero = Avx.CompareEqual(s, zero);
            invalidMask = Avx.Or(greaterOrEqualOne, equalZero);
        } while (Avx.MoveMask(invalidMask) != 0);

        // Calculate multiplier
        var multiplier = Avx.Divide(rsqrt2, Avx.Sqrt(s));

        // Generate two sets of normal random numbers
        if (count <= batchSize)
        {
            yield return Avx.Multiply(x, multiplier);
            count++;
        }
        if (count <= batchSize)
        {
            yield return Avx.Multiply(y, multiplier);
            count++;
        }
    }
}

static IEnumerable<T> ToEnumerable<T>(this Vector256<T> vector)
{
    for (int i = 0; i < Vector256<T>.Count; i++)
        yield return vector.GetElement(i);
}

int vectorCount = 5000;
var randomNumbers = GenerateGaussianVectorMarsaglia(vectorCount)
    .SelectMany(v => v.ToEnumerable()).ToArray();

// Some LINQ
var duplicateCount = randomNumbers
    .GroupBy(n => n)
    .Where(group => group.Count() > 1)
    .Sum(group => group.Count() - 1);
Console.WriteLine($"Duplicate numbers: {duplicateCount}");
// Some statistics

var mean = randomNumbers.Average();
var stdDev = Math.Sqrt(randomNumbers.Average(n => Math.Pow(n - mean, 2)));
Console.WriteLine($"Mean: {mean}, StdDev: {stdDev}");

### Conclusions

- Marsaglia; we have to check if number is hit or miss; in Ziggurat misses are rare
- Algorithms need to be scalable; from Vector256 to Vector512 for instance
- AVX should have random number generator
- AVX doesnt have logs and exponentials
    - We can probably try approximation; Taylor Series, Polynomial Fittings
- Vector of T using Vector512 where available, common APIs/operations there are limitied
- Intel's Short Vector Math Library

In [ ]:
static Vector256<double> LogApprox(Vector256<double> x)
{
    Vector256<double> one = Vector256.Create(1.0);
    Vector256<double> term = Avx2.Subtract(x, one);
    Vector256<double> result = term; // Linear approximation
    result = Avx2.Add(result, Avx2.Multiply(term, term)); // Higher-order terms if needed
    return result;
}

In [ ]:
using System.Runtime.Intrinsics;
using System.Runtime.Intrinsics.X86;

static Vector256<float> ComputeLog(Vector256<float> values)
{
    if (Avx2.IsSupported)
    {
        Vector256<float> one = Vector256.Create(1.0f);
        Vector256<float> result = Vector256.Create(0.0f);
        Vector256<float> current = values;

        for (int i = 0; i < 10; i++)
        {
            current = Avx.Multiply(current, Avx.Reciprocal(current));
            result = Avx.Add(result, Avx.Subtract(current, one));
        }

        return result;
    }
    else
        throw new PlatformNotSupportedException("AVX2 is not supported on this machine.");
}

static void PrintVector(Vector256<float> vector)
{
    float[] values = new float[8];
    vector.CopyTo(values);
    Console.WriteLine(string.Join(", ", values.Select(v => v.ToString("F6"))));
}

## Stochastic Thinking

__MIT Lectures: Introduction to Computational Thinking and Data Science__

- https://www.youtube.com/watch?v=-1BnXEwHUok Stochastic Thinking
- https://www.youtube.com/watch?v=6wUD_gp5WeE Random Walks
- https://www.youtube.com/watch?v=OgO1gpXSUzU Monte Carlo Simulation

### Geometric Brownian Motion (GBM)

- https://en.wikipedia.org/wiki/Call_option
- https://en.wikipedia.org/wiki/Option_style

- https://en.wikipedia.org/wiki/Geometric_Brownian_motion
We’ll focus on simulating random walks for asset prices and computing the payoff of a European call option

- Black-Scholes model with Monte Carlo simulation
    - https://en.wikipedia.org/wiki/Black%E2%80%93Scholes_model

__GBM Formula__

<img src=images/gbm-1.png width=700>

__Payoff__

<img src=images/gbm-2.png width=700>

In [ ]:
using System.Numerics;

Random random = new Random();

double GenerateGaussianNumber()  // Gaussian Random Number
{                                       // Gaussian distrubtion aka Normal distribution 
    // Box-Muller Transform
    double u1 = 1.0 - random.NextDouble();
    double u2 = 1.0 - random.NextDouble();
    return Math.Sqrt(-2.0 * Math.Log(u1)) * Math.Cos(2.0 * Math.PI * u2);
}

// Bitcoin option pricing parameters 
double initialPrice = 70000;    // Bitcoin initial price (S0)
double strikePrice = 100000;    // Strike price for the call option (K)
double riskFreeRate = 0.05;     // Risk-free rate (r)
double volatility = 0.7;        // Volatility of Bitcoin (σ)
double timeToMaturity = 1.0;    // Time to maturity (T) in years
int numberOfSimulations = 90000;
int vectorSize = Vector<double>.Count;

Vector<double> VectorExp(Vector<double> v)
{
    Span<double> result = stackalloc double[Vector<double>.Count];
    for (int i = 0; i < Vector<double>.Count; i++)
        result[i] = Math.Exp(v[i]);
    return new Vector<double>(result);
}

// Method to simulate the price of a European call option on Bitcoin
double SimulateOptionPrice()
{
    double sumPayoff = 0.0;
    var drift = riskFreeRate - 0.5 * volatility * volatility;
    
    for (int i = 0; i < numberOfSimulations; i += vectorSize)
    {
        var randomNumbers = new double[vectorSize];
        for (int j = 0; j < vectorSize; j++)
            randomNumbers[j] = GenerateGaussianNumber();

        // Vectors for the calculation
        var randVector = new Vector<double>(randomNumbers);
        var priceVector = new Vector<double>(Enumerable.Repeat(initialPrice, vectorSize).ToArray());
        var strikeVector = new Vector<double>(Enumerable.Repeat(strikePrice, vectorSize).ToArray());
        var driftVector = new Vector<double>(Enumerable.Repeat(drift, vectorSize).ToArray());
        var volVector = new Vector<double>(Enumerable.Repeat(volatility, vectorSize).ToArray());
        var timeVector = new Vector<double>(Enumerable.Repeat(timeToMaturity, vectorSize).ToArray());

        // Exponent for the GBM
        var exponent = driftVector * timeVector + 
            volVector * Vector.SquareRoot(timeVector) * randVector;

        // Final prices using exp(exponent)
        var finalPrices = priceVector * VectorExp(exponent);

        // Payoffs: max(S_T - K, 0)
        var payoffs = Vector.Max(finalPrices - strikeVector, new Vector<double>(new double[vectorSize]));

        // Sum up the payoffs
        sumPayoff += Vector.Sum(payoffs);
    }

    // Calculate the average payoff and discount to present value
    double averagePayoff = sumPayoff / numberOfSimulations;
    return averagePayoff * Math.Exp(-1 * riskFreeRate * timeToMaturity);
}

Console.WriteLine($"The estimated price of the Bitcoin call option is: ${SimulateOptionPrice():F2}");

# 📦 Tensors

Multi-Dimensional Arrays; essential for scientific computing, machine learning, and numerical analysis
- Unlike traditional arrays or jagged arrays, tensors provide a structured way to handle multi-dimensional data efficiently
- High-Performance
    - GPU / Hardware Acceleration

__Python Scene__
- Numpy has numpy.ndarray; not explicitly a tensor but functions similarly
- Pandas has pandas.DataFrame and pandas.Series that are based on NumPy arrays; not explicitly a tensor but functions similarly
    - JAX, CuPy etc have similar approach
- TensorFlow has tf.Tensor
    - GPU optimized; Object Oriented API 
- PyTorch has torch.Tensor
    - GPU optimized; Pythonic API

__.NET Scene__

- https://github.com/dotnet/TorchSharp Highly Active
    - libtorch based
    - Designed to offer Pytorch expereince
- https://github.com/SciSharp/NumSharp Last Nuget Release Feb of 2021
- https://github.com/SciSharp/Tensor.NET Last Nuget Release November of 2023
- System.Numerics.Tensors API; currently a nuget

__System.Numerics.Tensors__
- GPU / Hardware Acceleration
    - Efficient Memory Management; The library supports **span-based operations**
    - Works well with `Memory<T>` and `Span<T>` 
- (Will) provde Interoperability with AI/ML Libraries
    - ONNX, ML.NET, TorchSharp
- Cross-Platform support
- Ease of Adoption for Numerical Computing; beyond ML/AI

- https://www.nuget.org/packages/System.Numerics.Tensors; 2018
- https://learn.microsoft.com/en-us/dotnet/api/system.numerics.tensors
- https://github.com/dotnet/runtime/tree/main/src/libraries/System.Numerics.Tensors
    - Lets check the code; how its based on Vector

In [ ]:
#r "nuget: System.Numerics.Tensors"

- https://github.com/dotnet/core/blob/main/release-notes/9.0/preview/preview4/libraries.md

In [ ]:
using System.Numerics.Tensors;
// https://devblogs.microsoft.com/dotnet/announcing-dotnet-8-rc2

var movies = new[]
{
    new { Title="The Lion King", Embedding= new [] { 0.10022575f, -0.23998135f } },
    new { Title="Inception", Embedding= new [] { 0.10327095f, 0.2563685f } },
    new { Title="Toy Story", Embedding= new [] { 0.095857024f, -0.201278f } },
    new { Title="Pulp Fiction", Embedding= new [] { 0.106827796f, 0.21676421f } },
    new { Title="Shrek", Embedding= new [] { 0.09568083f, -0.21177962f } }
};
var queryEmbedding = new[] { 0.12217915f, -0.034832448f }; // we are looking towards somewhere like Toy Story

var top3Movies = movies.Select(movie =>
    (
        movie.Title,
        Similarity: TensorPrimitives.CosineSimilarity(queryEmbedding, movie.Embedding)
    ))
    .OrderByDescending(movies => movies.Similarity)
    .Take(3);

foreach (var movie in top3Movies)
    Console.WriteLine(movie);

- https://devblogs.microsoft.com/dotnet/introducing-tensor-for-multi-dimensional-machine-learning-and-ai-data 2017 / Outdated
- https://devblogs.microsoft.com/dotnet/announcing-ml-net-3-0/ 2023 November
- https://github.com/dotnet/runtime/issues/89639 [API Proposal]: Future of Numerics and AI
    - https://github.com/dotnet/runtime/issues/92219 Remaining .NET 8 work for TensorPrimitives
    - https://github.com/dotnet/runtime/issues/98323 Tensors in .NET